In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install hickle

     |████████████████████████████████| 49 kB 2.2 MB/s 
     |████████████████████████████████| 2.9 MB 7.0 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [ ]:
from __future__ import print_function

from keras import activations, initializers, constraints
from keras import regularizers
#from keras.engine import Layer
import keras.backend as K

import keras


class GraphLayer(keras.layers.Layer):

    def __init__(self,
                 step_num=1,
                 activation=None,
                 **kwargs):
        """Initialize the layer.
        :param step_num: Two nodes are considered as connected if they could be reached in `step_num` steps.
        :param activation: The activation function after convolution.
        :param kwargs: Other arguments for parent class.
        """
        self.supports_masking = True
        self.step_num = step_num
        self.activation = keras.activations.get(activation)
        self.supports_masking = True
        super(GraphLayer, self).__init__(**kwargs)

    def get_config(self):
        config = {
            'step_num': self.step_num,
            'activation': self.activation,
        }
        base_config = super(GraphLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def _get_walked_edges(self, edges, step_num):
        """Get the connection graph within `step_num` steps
        :param edges: The graph in single step.
        :param step_num: Number of steps.
        :return: The new graph that has the same shape with `edges`.
        """
        if step_num <= 1:
            return edges
        deeper = self._get_walked_edges(K.batch_dot(edges, edges), step_num // 2)
        if step_num % 2 == 1:
            deeper += edges
        return K.cast(K.greater(deeper, 0.0), K.floatx())

    def call(self, inputs, **kwargs):
        features, edges = inputs
        edges = K.cast(edges, K.floatx())
        if self.step_num > 1:
            edges = self._get_walked_edges(edges, self.step_num)
        outputs = self.activation(self._call(features, edges))
        return outputs

    def _call(self, features, edges):
        raise NotImplementedError('The class is not intended to be used directly.')


class GraphConv(GraphLayer):
    """Graph convolutional layer.
    h_i^{(t)} = \sigma \left ( \frac{ G_i^T (h_i^{(t - 1)} W + b)}{\sum G_i}  \right )
    """

    def __init__(self,
                 units,
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=None,
                 kernel_constraint=None,
                 use_bias=True,
                 bias_initializer='zeros',
                 bias_regularizer=None,
                 bias_constraint=None,
                 **kwargs):
        """Initialize the layer.
        :param units: Number of new states. If the input shape is (batch_size, node_num, feature_len), then the output
                      shape is (batch_size, node_num, units).
        :param kernel_initializer: The initializer of the kernel weight matrix.
        :param kernel_regularizer: The regularizer of the kernel weight matrix.
        :param kernel_constraint:  The constraint of the kernel weight matrix.
        :param use_bias: Whether to use bias term.
        :param bias_initializer: The initializer of the bias vector.
        :param bias_regularizer: The regularizer of the bias vector.
        :param bias_constraint: The constraint of the bias vector.
        :param kwargs: Other arguments for parent class.
        """
        self.units = units
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.kernel_constraint = keras.constraints.get(kernel_constraint)
        self.use_bias = use_bias
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)
        self.bias_constraint = keras.constraints.get(bias_constraint)

        self.W, self.b = None, None
        super(GraphConv, self).__init__(**kwargs)

    def get_config(self):
        config = {
            'units': self.units,
            'kernel_initializer': keras.initializers.serialize(self.kernel_initializer),
            'kernel_regularizer': keras.regularizers.serialize(self.kernel_regularizer),
            'kernel_constraint': keras.constraints.serialize(self.kernel_constraint),
            'use_bias': self.use_bias,
            'bias_initializer': keras.initializers.serialize(self.bias_initializer),
            'bias_regularizer': keras.regularizers.serialize(self.bias_regularizer),
            'bias_constraint': keras.constraints.serialize(self.bias_constraint),
        }
        base_config = super(GraphConv, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        feature_dim = input_shape[0][2]
        self.W = self.add_weight(
            shape=(feature_dim, self.units),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='{}_W'.format(self.name),
        )
        if self.use_bias:
            self.b = self.add_weight(
                shape=(self.units,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='{}_b'.format(self.name),
            )
        super(GraphConv, self).build(input_shape)

    def compute_output_shape(self, input_shape):
        return input_shape[0][:2] + (self.units,)

    # def compute_mask(self, inputs, mask=None):
    #     return mask[0]

    def _call(self, features, edges):
        features = K.dot(features, self.W)
        if self.use_bias:
            features += self.b
        if self.step_num > 1:
            edges = self._get_walked_edges(edges, self.step_num)
        return K.batch_dot(K.permute_dimensions(edges, (0, 2, 1)), features) #\
           # / (K.sum(edges, axis=2, keepdims=True) + K.epsilon())


class GraphPool(GraphLayer):

    def compute_output_shape(self, input_shape):
        return input_shape

    def compute_mask(self, inputs, mask=None):
        return mask[0]


class GraphMaxPool(GraphPool):

    NEG_INF = -1e38

    def _call(self, features, edges):
        node_num = K.shape(features)[1]
        features = K.tile(K.expand_dims(features, axis=1), K.stack([1, node_num, 1, 1])) \
            + K.expand_dims((1.0 - edges) * self.NEG_INF, axis=-1)
        return K.max(features, axis=2)


class GraphAveragePool(GraphPool):

    def _call(self, features, edges):
        return K.batch_dot(K.permute_dimensions(edges, (0, 2, 1)), features) \
            / (K.sum(edges, axis=2, keepdims=True) + K.epsilon())

In [ ]:
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input,InputLayer,Multiply,ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D,Conv1D,MaxPooling1D
from keras.layers import Dense,Activation,Dropout,Flatten,Concatenate
from keras.layers import BatchNormalization
from keras.layers import Lambda
from keras.layers import Dropout,GlobalMaxPooling1D,GlobalAveragePooling1D
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
#from layers.graph import GraphLayer,GraphConv




class KerasMultiSourceGCNModel(object):
    def __init__(self,use_mut,use_gexp,use_methy,use_cnv,regr=True):#
        self.use_mut = use_mut
        self.use_gexp = use_gexp
        self.use_methy = use_methy
        self.use_cnv = use_cnv
        self.regr = regr
    def createMaster(self,drug_dim,mutation_dim,gexpr_dim,methy_dim,cnv_dim,units_list,use_relu=True,use_bn=True,use_GMP=False):
        drug_feat_input = Input(shape=(None,drug_dim),name='drug_feat_input')#drug_dim=75
        drug_adj_input = Input(shape=(None,None),name='drug_adj_input')

        mutation_input = Input(shape=(mutation_dim,),name='mutation_feat_input')
        gexpr_input = Input(shape=(gexpr_dim,),name='gexpr_feat_input')
        methy_input = Input(shape=(methy_dim,),name='methy_feat_input')
        cnv_input = Input(shape=(cnv_dim,),name='cnv_feat_input')

        #drug feature with GCN
        GCN_layer = GraphConv(units=units_list[0],step_num=1)([drug_feat_input,drug_adj_input])
        if use_relu:
            GCN_layer = Activation('relu')(GCN_layer)
        else:
            GCN_layer = Activation('tanh')(GCN_layer)
        if use_bn:
            GCN_layer = BatchNormalization()(GCN_layer)
        GCN_layer = Dropout(0.1)(GCN_layer)

        for i in range(len(units_list)-1):
            GCN_layer = GraphConv(units=units_list[i+1],step_num=1)([GCN_layer,drug_adj_input])
            if use_relu:
                GCN_layer = Activation('relu')(GCN_layer)
            else:
                GCN_layer = Activation('tanh')(GCN_layer)
            if use_bn:
                GCN_layer = BatchNormalization()(GCN_layer)
            GCN_layer = Dropout(0.1)(GCN_layer)

        GCN_layer = GraphConv(units=100,step_num=1)([GCN_layer,drug_adj_input])
        if use_relu:
            GCN_layer = Activation('relu')(GCN_layer)
        else:
            GCN_layer = Activation('tanh')(GCN_layer)
        if use_bn:
            GCN_layer = BatchNormalization()(GCN_layer)
        GCN_layer = Dropout(0.1)(GCN_layer)
        #global pooling
        if use_GMP:
            x_drug = GlobalMaxPooling1D()(GCN_layer)
        else:
            x_drug = GlobalAveragePooling1D()(GCN_layer)

        x = x_drug
        if self.use_mut:
            x = Concatenate()([x,mutation_input])
        if self.use_gexp:
            x = Concatenate()([x,gexpr_input])
        if self.use_methy:
            x = Concatenate()([x,methy_input])
        if self.use_cnv:
            x = Concatenate()([x,cnv_input])

        #x = Concatenate()([x_mut,x_drug,x_gexpr,x_methy])
        x = Dense(300,activation = 'tanh')(x)
        x = Dropout(0.1)(x)
        x = Lambda(lambda x: K.expand_dims(x,axis=-1))(x)
        x = Lambda(lambda x: K.expand_dims(x,axis=1))(x)
        x = Conv2D(filters=30, kernel_size=(1,150),strides=(1, 1), activation = 'relu',padding='valid')(x)
        x = MaxPooling2D(pool_size=(1,2))(x)
        x = Conv2D(filters=10, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
        x = MaxPooling2D(pool_size=(1,3))(x)
        x = Conv2D(filters=5, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
        x = MaxPooling2D(pool_size=(1,3))(x)
        x = Dropout(0.1)(x)
        x = Flatten()(x)
        x = Dropout(0.2)(x)


        if self.regr:
            output = Dense(1,name='output')(x)
        else:
            output = Dense(1,activation = 'sigmoid',name='output')(x)
        model  = Model(inputs=[drug_feat_input,drug_adj_input,mutation_input,gexpr_input,methy_input,cnv_input],outputs=output)
        return model

In [ ]:
import argparse
import random,os,sys
import numpy as np
import csv
from scipy import stats
import time
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
import pandas as pd
import keras.backend as K
from keras.models import Model, Sequential
from keras.models import load_model
from keras.layers import Input,InputLayer,Multiply,ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense,Activation,Dropout,Flatten,Concatenate
from keras.layers import BatchNormalization
from keras.layers import Lambda
from keras import optimizers,utils
from keras.constraints import max_norm
from keras import regularizers
from keras.callbacks import ModelCheckpoint,Callback,EarlyStopping,History
from tensorflow.keras.optimizers import Adam, SGD
from keras.models import model_from_json
import tensorflow as tf
from sklearn.metrics import average_precision_score
from scipy.stats import pearsonr

import hickle as hkl
import scipy.sparse as sp


####################################Constants Settings###########################

DPATH = '/content/drive/MyDrive/Datasets/ModelAGCN'
Drug_feature_file = '/content/drive/MyDrive/Datasets/103_smiles'
Genomic_mutation_file = '/content/drive/MyDrive/Datasets/dae_mutation.xlsx'
Cancer_response_exp_file = '/content/drive/MyDrive/Datasets/103BRCADrugs(normalized).xlsx'
Gene_expression_file = '/content/drive/MyDrive/Datasets/dae_expression.xlsx'
Methylation_file = '/content/drive/MyDrive/Datasets/dae_meth.xlsx'
cnv_file = '/content/drive/MyDrive/Datasets/dae_cnv.xlsx'
Max_atoms = 100
model_suffix = 'abc'

def MetadataGenerate(Genomic_mutation_file,Drug_feature_file,Gene_expression_file,Methylation_file,filtered):
    # #drug_id --> pubchem_id
    # reader = csv.reader(open(Drug_info_file,'r'))
    # rows = [item for item in reader]
    # drugid2pubchemid = {item[0]:item[5] for item in rows if item[5].isdigit()}

    # #map cellline --> cancer type
    # cellline2cancertype ={}
    # for line in open(Cell_line_info_file).readlines()[1:]:
    #     cellline_id = line.split('\t')[1]
    #     TCGA_label = line.strip().split('\t')[-1]
    #     #if TCGA_label in TCGA_label_set:
    #     cellline2cancertype[cellline_id] = TCGA_label

    #load demap cell lines genomic mutation features
    mutation_feature = pd.read_excel(Genomic_mutation_file, header=0, index_col=0)
    cell_line_name_set = list(mutation_feature.index)

    # load drug features
    drug_name_set = []
    drug_feature = {}
    for each in os.listdir(Drug_feature_file):
        drug_name_set.append(each.split('.')[0])
        feat_mat,adj_list,degree_list = hkl.load('%s/%s'%(Drug_feature_file,each))
        drug_feature[each.split('.')[0]] = [feat_mat,adj_list,degree_list]

    #load gene expression faetures
    gexpr_feature = pd.read_excel(Gene_expression_file, header=0, index_col=0)

    #load methylation
    methylation_feature = pd.read_excel(Methylation_file, header=0, index_col=0)

    cnv_feature = pd.read_excel(cnv_file, header=0, index_col=0)

    assert methylation_feature.shape[0]==gexpr_feature.shape[0]==mutation_feature.shape[0]

    experiment_data = pd.read_excel(Cancer_response_exp_file, header=0, index_col=0)
    print(experiment_data)
    #filter experiment data
    #drug_match_list=[item for item in experiment_data.index if item.split(':')[1] in drugid2pubchemid.keys()]
    experiment_data_filtered = experiment_data.copy()

    data_idx = []
    for each_cellline in experiment_data_filtered.index:
      for each_drug in experiment_data_filtered.columns:
        ln_IC50 = float(experiment_data_filtered.loc[each_cellline, each_drug])
        data_idx.append((each_cellline, each_drug, ln_IC50))

    nb_celllines = len(set([item[0] for item in data_idx]))
    nb_drugs = len(set([item[1] for item in data_idx]))
    print('%d instances across %d cell lines and %d drugs were generated.'%(len(data_idx),nb_celllines,nb_drugs))
    return mutation_feature, drug_feature, gexpr_feature, methylation_feature, cnv_feature, data_idx

#split into training and test set
def DataSplit(data_idx,ratio = 0.95):
    data_train_idx,data_test_idx = [], []
    train_list = random.sample(data_idx,int(ratio*len(data_idx)))
    test_list = [item for item in data_idx if item not in train_list]
    data_train_idx += train_list
    data_test_idx += test_list
    return data_train_idx,data_test_idx

def NormalizeAdj(adj):
    adj = adj + np.eye(adj.shape[0])
    d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0).toarray()
    a_norm = adj.dot(d).transpose().dot(d)
    return a_norm

def random_adjacency_matrix(n):
    matrix = [[random.randint(0, 1) for i in range(n)] for j in range(n)]
    # No vertex connects to itself
    for i in range(n):
        matrix[i][i] = 0
    # If i is connected to j, j is connected to i
    for i in range(n):
        for j in range(n):
            matrix[j][i] = matrix[i][j]
    return matrix

def CalculateGraphFeat(feat_mat,adj_list):
    assert feat_mat.shape[0] == len(adj_list)
    feat = np.zeros((Max_atoms,feat_mat.shape[-1]),dtype='float32')
    adj_mat = np.zeros((Max_atoms,Max_atoms),dtype='float32')

    feat[:feat_mat.shape[0],:] = feat_mat
    for i in range(len(adj_list)):
        nodes = adj_list[i]
        for each in nodes:
            adj_mat[i,int(each)] = 1
    assert np.allclose(adj_mat,adj_mat.T)
    adj_ = adj_mat[:len(adj_list),:len(adj_list)]
    adj_2 = adj_mat[len(adj_list):,len(adj_list):]
    norm_adj_ = NormalizeAdj(adj_)
    norm_adj_2 = NormalizeAdj(adj_2)
    adj_mat[:len(adj_list),:len(adj_list)] = norm_adj_
    adj_mat[len(adj_list):,len(adj_list):] = norm_adj_2
    return [feat,adj_mat]

def FeatureExtract(data_idx,drug_feature,mutation_feature,gexpr_feature,methylation_feature, cnv_feature):

    nb_instance = len(data_idx)
    nb_mutation_feature = mutation_feature.shape[1]
    nb_gexpr_features = gexpr_feature.shape[1]
    nb_methylation_features = methylation_feature.shape[1]
    nb_cnv_features = cnv_feature.shape[1]

    drug_data = [[] for item in range(nb_instance)]
    mutation_data = np.zeros((nb_instance, nb_mutation_feature),dtype='float32')
    gexpr_data = np.zeros((nb_instance,nb_gexpr_features),dtype='float32')
    cnv_data = np.zeros((nb_instance,nb_cnv_features),dtype='float32')
    methylation_data = np.zeros((nb_instance, nb_methylation_features),dtype='float32')
    target = np.zeros(nb_instance,dtype='float32')

    for idx in range(nb_instance):
        cell_line_name, drug_name, ln_IC50 = data_idx[idx]
        #modify
        feat_mat,adj_list,_ = drug_feature[str(drug_name)]
        #fill drug data,padding to the same size with zeros
        drug_data[idx] = CalculateGraphFeat(feat_mat,adj_list)
        #randomlize X A
        mutation_data[idx,:] = mutation_feature.loc[cell_line_name].values
        gexpr_data[idx,:] = gexpr_feature.loc[cell_line_name].values
        cnv_data[idx,:] = cnv_feature.loc[cell_line_name].values
        methylation_data[idx,:] = methylation_feature.loc[cell_line_name].values
        target[idx] = ln_IC50
    return drug_data,mutation_data,gexpr_data,methylation_data,target,cnv_data



class MyCallback(Callback):
    def __init__(self,validation_data,patience):
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        self.best_weight = None
        self.patience = patience
    def on_train_begin(self,logs={}):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        return
    def on_train_end(self, logs={}):
        self.model.set_weights(self.best_weight)
        self.model.save('../checkpoint/Best_%s.h5'%model_suffix)
        if self.stopped_epoch > 0 :
            print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
        pcc_val = pearsonr(self.y_val, y_pred_val[:,0])[0]
        print ('pcc-val: %s'%str(round(pcc_val,4)))
        if pcc_val > self.best:
            self.best = pcc_val
            self.wait = 0
            self.best_weight = self.model.get_weights()
        else:
            self.wait+=1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
        return




def ModelTraining(model,X_drug_data_train,X_mutation_data_train,X_gexpr_data_train,X_methylation_data_train,X_cnv_data_train,Y_train,validation_data,nb_epoch=100):
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer = optimizer,loss='mean_squared_error',metrics=['mse'])
    EarlyStopping(monitor='val_loss',patience=5)
    callbacks = [ModelCheckpoint('../checkpoint/best_%s.h5'%model_suffix,monitor='val_loss',save_best_only=False, save_weights_only=False),
                 MyCallback(validation_data=validation_data,patience=10)]
    X_drug_feat_data_train = [item[0] for item in X_drug_data_train]
    X_drug_adj_data_train = [item[1] for item in X_drug_data_train]
    X_drug_feat_data_train = np.array(X_drug_feat_data_train)#nb_instance * Max_stom * feat_dim
    X_drug_adj_data_train = np.array(X_drug_adj_data_train)#nb_instance * Max_stom * Max_stom
    #validation data
    model.fit(x = [X_drug_feat_data_train,X_drug_adj_data_train,X_mutation_data_train,X_gexpr_data_train,X_methylation_data_train,X_cnv_data_train], y=Y_train, batch_size=64, epochs=nb_epoch, validation_split=0, callbacks=callbacks)
    return model


def ModelEvaluate(model,X_drug_data_test,X_mutation_data_test,X_gexpr_data_test,X_methylation_data_test,X_cnv_data_test,Y_test,file_path):
    X_drug_feat_data_test = [item[0] for item in X_drug_data_test]
    X_drug_adj_data_test = [item[1] for item in X_drug_data_test]
    X_drug_feat_data_test = np.array(X_drug_feat_data_test)#nb_instance * Max_stom * feat_dim
    X_drug_adj_data_test = np.array(X_drug_adj_data_test)#nb_instance * Max_stom * Max_stom
    Y_pred = model.predict([X_drug_feat_data_test,X_drug_adj_data_test,X_mutation_data_test,X_gexpr_data_test,X_methylation_data_test,X_cnv_data_test])
    overall_pcc = pearsonr(Y_pred[:,0],Y_test)[0]
    print("The overall Pearson's correlation is %.4f."%overall_pcc)


def main():
    random.seed(0)
    mutation_feature, drug_feature,gexpr_feature,methylation_feature, cnv_feature, data_idx = MetadataGenerate(Genomic_mutation_file,Drug_feature_file,Gene_expression_file,Methylation_file,False)
    data_train_idx,data_test_idx = DataSplit(data_idx)
    #Extract features for training and test
    X_drug_data_train, X_mutation_data_train, X_gexpr_data_train, X_methylation_data_train, Y_train, X_cnv_data_train = FeatureExtract(data_train_idx,drug_feature,mutation_feature,gexpr_feature,methylation_feature, cnv_feature)
    X_drug_data_test,X_mutation_data_test,X_gexpr_data_test,X_methylation_data_test,Y_test, X_cnv_data_test = FeatureExtract(data_test_idx,drug_feature,mutation_feature,gexpr_feature,methylation_feature, cnv_feature)

    X_drug_feat_data_test = [item[0] for item in X_drug_data_test]
    X_drug_adj_data_test = [item[1] for item in X_drug_data_test]
    X_drug_feat_data_test = np.array(X_drug_feat_data_test)#nb_instance * Max_stom * feat_dim
    X_drug_adj_data_test = np.array(X_drug_adj_data_test)#nb_instance * Max_stom * Max_stom

    validation_data = [[X_drug_feat_data_test,X_drug_adj_data_test,X_mutation_data_test,X_gexpr_data_test,X_methylation_data_test, X_cnv_data_test],Y_test]
    model = KerasMultiSourceGCNModel(1, 1, 1, 1).createMaster(X_drug_data_train[0][0].shape[-1],X_mutation_data_train.shape[-1],X_gexpr_data_train.shape[-1],X_methylation_data_train.shape[-1],X_cnv_data_train.shape[-1],[256,256,256],1,1,0)
    print('Begin training...')
    model = ModelTraining(model,X_drug_data_train,X_mutation_data_train,X_gexpr_data_train,X_methylation_data_train,X_cnv_data_train,Y_train,validation_data,nb_epoch=500)
    ModelEvaluate(model,X_drug_data_test,X_mutation_data_test,X_gexpr_data_test,X_methylation_data_test,X_cnv_data_test,Y_test,'%s/AGCN_%s.log'%(DPATH,model_suffix))
    model.save('/content/drive/MyDrive/Datasets/ModelAGCN')

if __name__=='__main__':
    main()

            Camptothecin  Vinblastine  Cisplatin  ...    GSK591     VE821   AT13148
cell_name                                         ...                              
AU565           0.187431     0.379760   0.260516  ...  0.000000  0.222712  0.033434
BT-20           0.293815     0.492720   0.448197  ...  0.513782  0.507204  0.489218
BT-474          1.000000     0.447836   0.938684  ...  0.681941  0.754417  0.794431
BT-483          0.825094     1.000000   0.968622  ...  1.000000  1.000000  1.000000
BT-549          0.418418     0.437978   0.567320  ...  0.543242  0.450515  0.184297
CAL-120         0.428605     0.515088   0.925781  ...  0.738343  0.846053  0.703332
CAL-148         0.000000     0.047356   0.020386  ...  0.253041  0.168968  0.000000
CAL-51          0.141218     0.111984   0.187146  ...  0.262197  0.047359  0.269039
CAL-85-1        0.425566     0.471763   0.525818  ...  0.543045  0.561333  0.678803
CAMA-1          0.491949     0.457263   0.387248  ...  0.527947  0.483412  0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/500
71/71 [==============================] - 29s 375ms/step - loss: 0.0684 - mse: 0.0684
Epoch 2/500
71/71 [==============================] - 26s 371ms/step - loss: 0.0485 - mse: 0.0485
Epoch 3/500
71/71 [==============================] - 26s 360ms/step - loss: 0.0447 - mse: 0.0447
Epoch 4/500
71/71 [==============================] - 26s 361ms/step - loss: 0.0422 - mse: 0.0422
Epoch 5/500
71/71 [==============================] - 26s 372ms/step - loss: 0.0411 - mse: 0.0411
Epoch 6/500
71/71 [==============================] - 25s 358ms/step - loss: 0.0381 - mse: 0.0381
Epoch 7/500
71/71 [==============================] - 25s 359ms/step - loss: 0.0391 - mse: 0.0391
Epoch 8/500
71/71 [==============================] - 25s 359ms/step - loss: 0.0357 - mse: 0.0357
Epoch 9/500
71/71 [==============================] - 25s 359ms/step - loss: 0.0363 - mse: 0.0363
Epoch 10/500
71/71 [==============================] - 25s 358ms/step - loss: 0.0350 - mse: 0.0350
Epoch 11/500
71/71 [=========